In [ ]:
from vexa import VexaAPI
from search import SearchAssistant

vexa = VexaAPI()


User information retrieved successfully.


In [2]:
user_id = vexa.user_id
user_name = vexa.user_name


In [3]:
search_assistant = SearchAssistant()


In [8]:

query = "list vexa users"

# Use the chat method
async for response in search_assistant.chat(user_id, query, user_name):
    if isinstance(response, dict):
        print(f"Thread ID: {response['thread_id']}")
        print(f"Linked Output: {response['linked_output']}")
    else:
        print(response, end='', flush=True)


Based on the provided context, the following users are mentioned as using Vexa:

1. Dmitriy Grankin [1]
2. Sergey Ryabenko [1][2]
3. Sergio Goriachev [17]

These users are mentioned as using or testing Vexa in various meetings, including discussions about its functionality, testing, and potential applications. [1][2][17]Thread upserted with id 93091d3f-4ce7-4495-94e9-75ee6265ac6d
Thread ID: 93091d3f-4ce7-4495-94e9-75ee6265ac6d
Linked Output: Based on the provided context, the following users are mentioned as using Vexa:

1. Dmitriy Grankin [1](https://dashboard.vexa.ai/#e2864cf9-f8d5-4a22-85bc-d07557b634ee)
2. Sergey Ryabenko [1](https://dashboard.vexa.ai/#e2864cf9-f8d5-4a22-85bc-d07557b634ee) [2](https://dashboard.vexa.ai/#2b309a6a-7d85-4a1c-aaa8-4de0c94a03c3)
3. Sergio Goriachev [17](https://dashboard.vexa.ai/#2647ed9b-411e-46dc-ba53-c08ef2887479)

These users are mentioned as using or testing Vexa in various meetings, including discussions about its functionality, testing, and poten

In [4]:
response

NameError: name 'response' is not defined

In [ ]:
self = search_assistant

In [ ]:
if thread_id:
    thread = self.thread_manager.get_thread(thread_id)
    if not thread:
        raise ValueError(f"Thread with id {thread_id} not found")
    messages = thread.messages
    thread_name = thread.thread_name
else:
    messages = []
    thread_name = None

In [5]:
from vector_search import VectorSearch,build_context_string
from datetime import datetime
from core import system_msg, user_msg
from pydantic_models import QueryPlan
import pandas as pd

In [6]:
from qdrant_client import QdrantClient, models

In [7]:
vector_search = VectorSearch()

In [8]:
collection_name = vector_search.collection_name

In [9]:
filter_conditions = [
    models.FieldCondition(
        key="type",
        match=models.MatchValue(value="summary")
    )
]

if user_id:
    filter_conditions.append(models.FieldCondition(key="user_id", match=models.MatchValue(value=user_id)))


In [10]:
collection_name

'transcript_collection'

In [11]:
r = vector_search.qdrant_client.scroll(
            collection_name=collection_name,
            scroll_filter=models.Filter(must=filter_conditions),
            limit=10000,
            with_payload=True,
            with_vectors=False
        )[0]

In [12]:
len(r)


189

In [13]:
vector_search

In [14]:
last_n_meetings = 10000

In [15]:
summaries = vector_search.get_summaries(user_id=user_id, user_name=user_name)

In [16]:
summaries

[{'content': 'The meeting focused on integrating Google Analytics into the project and updating the UI/UX design. Dmitriy Grankin highlighted the urgency of setting up analytics and the need for user feedback on the dashboard. Okafor Ikenna discussed the requirements for UI/UX design work. The team plans to continue collaboration on these tasks and address technical issues with the codebase.',
  'meeting_name': 'Technical Team Meeting on Project Updates',
  'start_datetime': 1725009895,
  'speakers': ['Okafor Ikenna',
   'Olga Nemirovskaya',
   'Dmitriy Grankin',
   'Sergey Ryabenko'],
  'meeting_session_id': '6baf67cc-a30f-45ae-be59-255b011415ee',
  'user_id': 'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210',
  'user_name': 'Dmitriy Grankin',
  'source_type': 'meeting',
  'type': 'summary'},
 {'content': 'The meeting involved Andrey Pisankin, Olga Nemirovskaya, Dmitriy Grankin, and Sergey Ryabenko discussing updates to a project, focusing on code changes, troubleshooting issues, and deployment 

In [17]:
def build_context_string(summaries, points_by_meeting=None, only_summaries=False, include_all_summaries=True):
    context = []
    meeting_counter = 1
    meeting_ids = []
    for summary in summaries:
        meeting_id = summary['meeting_session_id']
        if include_all_summaries or (points_by_meeting and meeting_id in points_by_meeting):
            start_datetime = pd.to_datetime(datetime.fromtimestamp(summary['start_datetime']))
            context.append(f"# [{meeting_counter}] {summary['meeting_name']} ({start_datetime.strftime('%Y-%m-%d %H:%M')}) participants: {summary['speakers']}\n{summary['content']}")
            
            if not only_summaries and points_by_meeting and meeting_id in points_by_meeting:
                context.append("## Related Quotes:")
                for point in points_by_meeting[meeting_id]:
                    context.append(f"{point['qoutes']}")
            
            context.append("\n---\n")
            meeting_ids.append(meeting_id)
            meeting_counter += 1

    # Join context
    full_context = "\n".join(context)
    return full_context, meeting_ids

In [18]:
print(build_context_string(summaries,only_summaries=True)[0])

# [1] Technical Team Meeting on Project Updates (2024-08-30 12:24) participants: ['Okafor Ikenna', 'Olga Nemirovskaya', 'Dmitriy Grankin', 'Sergey Ryabenko']
The meeting focused on integrating Google Analytics into the project and updating the UI/UX design. Dmitriy Grankin highlighted the urgency of setting up analytics and the need for user feedback on the dashboard. Okafor Ikenna discussed the requirements for UI/UX design work. The team plans to continue collaboration on these tasks and address technical issues with the codebase.

---

# [2] Technical Project Update Meeting (2024-08-30 12:47) participants: ['Andrey Pisankin', 'Olga Nemirovskaya', 'Dmitriy Grankin', 'Sergey Ryabenko']
The meeting involved Andrey Pisankin, Olga Nemirovskaya, Dmitriy Grankin, and Sergey Ryabenko discussing updates to a project, focusing on code changes, troubleshooting issues, and deployment of a dashboard. Key topics included cookie management and Google Analytics integration.

---

# [3] Marketing St

In [19]:

now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
planner_prompt = """Think step by step. 
                    1. Understand the provided context in the form of meeting summaries that user has been part of.
                    2. Think of what is the problem or objective behind the user request.
                    3. Think of what are the most relevant meetings to the user request given the objective and context.
                    4. and create many (5+) requests to the search system which will be used to find similar meetings to the user request.
                    5. think of what might be the content inside the meetings that will be relevant to the user request.
                    6. create queries to the search system which will be used to find similar meetings to the user request.
                    requests should have at least 125 char each and at least 2 sentences. Extract relevant entities and concepts directly fromthe context and in the requests to get the most relevant meetings."""


In [20]:

plan = await QueryPlan.call([
    system_msg(f"""{planner_prompt}.
                    Now is {now}.
                    Write queries based on user request and general context you have, don't use start and end if timing is not obvious from the user query.
                """),
    user_msg(f'general context: last {last_n_meetings} meeting summaries in ascending order: {build_context_string(summaries,only_summaries=True)[0]}'),
    user_msg(f'user request: {query}.')
])

# Extract queries from the plan
queries = plan[0].model_dump()['queries']


NameError: name 'query' is not defined

In [21]:
queries

NameError: name 'queries' is not defined

In [22]:

query_ = query
queries = await self.analyzer.generate_search_queries(query_, user_id=user_id, user_name=user_name)

NameError: name 'query' is not defined

In [23]:
queries

NameError: name 'queries' is not defined

In [ ]:


summaries = self.analyzer.get_summaries(user_id=user_id, user_name=user_name)
full_context, meeting_ids = await self.analyzer.build_context(queries, summaries, include_all_summaries=False, user_id=user_id, user_name=user_name,k=40)

pref = "Based on the following context, answer the question:" if len(messages) == 0 else "Follow-up request:"
user_info = f"The User is {user_name}"
messages_context = [
    system_msg(self.prompts.perplexity + f'. {user_info}'), 
    user_msg(f"Context:\n{full_context}"),
] + messages + [user_msg(f"{pref} {query}. ALways suplay references to meetings as [1][2][3] etc.")]

# Use the provided model if given, otherwise use the default model
model_to_use = model or self.model

output = ""
async for chunk in generic_call_(messages_context, model=model_to_use, temperature=temperature, streaming=True):
    output += chunk
    yield chunk

# Create linked_output before appending messages
indexed_meetings = self.get_indexed_meetings(meeting_ids, self.parse_refs(output))
url_dict = {k: f'https://dashboard.vexa.ai/#{v}' for k, v in indexed_meetings.items()}
linked_output = self.embed_links(output, url_dict)

messages.append(user_msg(query))
messages.append(assistant_msg(linked_output))

# New code for indexing meetings and embedding links
indexed_meetings = self.get_indexed_meetings(meeting_ids, self.parse_refs(output))
url_dict = {k: f'https://dashboard.vexa.ai/#{v}' for k, v in indexed_meetings.items()}
linked_output = self.embed_links(output, url_dict)

if not thread_id:
    messages_str = ';'.join([m.content for m in messages if m.role == 'user'])
    thread_name = await ThreadName.call([user_msg(messages_str)])
    thread_name = thread_name[0].thread_name
    thread_id = self.thread_manager.upsert_thread(user_id=user_id, thread_name=thread_name, messages=messages)
else:
    self.thread_manager.upsert_thread(user_id = user_id, messages = messages,thread_id=thread_id)

# Instead of yielding the entire SearchResult, we only yield the thread_id and linked_output
yield {
    "thread_id": thread_id,
    "linked_output": linked_output
}

In [24]:
from search import SearchAssistant
import random

search_assistant = SearchAssistant()

In [25]:
threads = search_assistant.get_user_threads(user_id)

In [33]:
thread_id = threads[0]['thread_id']

In [34]:
messages = search_assistant.get_messages_by_thread_id(thread_id)

In [35]:
messages[0]

Msg(role='user', content='list a few speakers', stage=None, service_content=None)

In [128]:
import re

In [131]:

# Clean assistant messages of dashboard URLs
cleaned_messages = []
for message in messages:
    if message.role == 'assistant':
        cleaned_content = re.sub(r'https://dashboard\.vexa\.ai/#[^\s\]]+', '', message.content)
        message.content = cleaned_content
        cleaned_messages.append(message)
    else:
        cleaned_messages.append(message)
messages = cleaned_messages

In [132]:
messages

[Msg(role='user', content='list 3 participants', stage=None),
 Msg(role='assistant', content="Based on the context, here are three participants mentioned in the meetings:\n\n1. Dmitriy Grankin [1]( [2]( [3](\n2. Olga Nemirovskaya [2]( [3](\n3. Sergey Ryabenko [2]( [3](\n\nLet me know if you'd like me to list more participants!", stage=None)]